In [1]:
cd /class/datamine/corporate/farfetch/

/class/datamine/corporate/farfetch


In [1]:
import json
import pandas as pd
from collections import Counter
import numpy as np

In [4]:
f=open('meta_all_129927.json',)

In [5]:
data=json.load(f)

In [6]:
file=pd.read_json('meta_all_129927.json')
#can test with small dataset
#file=pd.read_json('300portion_data.json')

In [9]:
def unique(list1):
    x = np.array(list1)
    return(np.unique(x))
uniq=[]
for i in file['attr']:
    #print(unique(i))
    if unique(i) is not None:
        for j in unique(i):
            uniq.append(j) 
freq=Counter(uniq,reverse=True)
ranking=sorted(freq.items(), key=lambda pair: pair[1], reverse=True)

In [19]:
ranking[:50]

[('nah', 126751),
 ('cotton', 16736),
 ('cut', 16391),
 ('soft', 13036),
 ('look', 12023),
 ('style', 11671),
 ('sleeve', 11244),
 ('fit', 9957),
 ('stretch', 9429),
 ('classic', 8422),
 ('comfort', 8207),
 ('print', 8086),
 ('leather', 8044),
 ('knit', 7932),
 ('waist', 7577),
 ('hem', 7303),
 ('blend', 7232),
 ('day', 6479),
 ('easy', 6443),
 ('front', 6417),
 ('stripe', 6407),
 ('strap', 6381),
 ('high', 6205),
 ('lightweight', 5755),
 ('button', 5711),
 ('shoulder', 5420),
 ('neck', 5410),
 ('leg', 5192),
 ('logo', 4944),
 ('slim', 4937),
 ('heel', 4838),
 ('color', 4810),
 ('signature', 4774),
 ('neckline', 4771),
 ('denim', 4769),
 ('wool', 4757),
 ('modern', 4749),
 ('cool', 4607),
 ('one-piece', 4544),
 ('collar', 4416),
 ('lace', 4415),
 ('side', 4340),
 ('perfect', 4194),
 ('pocket', 4118),
 ('silk', 3988),
 ('cuff', 3967),
 ('pattern', 3954),
 ('floral', 3951),
 ('toe', 3889),
 ('detail', 3842)]

In [ ]:
#keep only top 50 attributes for the dataset


#split test-train


In [2]:
import torch
import torch.nn as nn
class Resnext50(nn.Module):

    def __init__(self, n_classes):

        super().__init__()

        resnet = models.resnext50_32x4d(pretrained=True)

        resnet.fc = nn.Sequential(

            nn.Dropout(p=0.2),

            nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)

        )

        self.base_model = resnet

        self.sigm = nn.Sigmoid()

    def forward(self, x):

        return self.sigm(self.base_model(x))

# Initialize the model

model = Resnext50(len(train_data))

# Switch model to the training mode

model.train()


NameError: name 'train_dataset' is not defined

In [ ]:
#parameter
batch_size = 32
max_epoch_number = 35
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Use threshold to define predicted labels and invoke sklearn's metrics with different averaging strategies.
def calculate_metrics(pred, target, threshold=0.5):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }

In [44]:
#training process
epoch = 0
iteration = 0
while True:
    batch_losses = []
    for imgs, targets in train_dataloader:
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()

        model_result = model(imgs)
        loss = criterion(model_result, targets.type(torch.float))

        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)

        if iteration % test_freq == 0:
            model.eval()
            with torch.no_grad():
                model_result = []
                targets = []
                for imgs, batch_targets in test_dataloader:
                    imgs = imgs.to(device)
                    model_batch_result = model(imgs)
                    model_result.extend(model_batch_result.cpu().numpy())
                    targets.extend(batch_targets.cpu().numpy())

            result = calculate_metrics(np.array(model_result), np.array(targets))
            print("epoch:{:2d} iter:{:3d} test: "
                  "micro f1: {:.3f} "
                  "macro f1: {:.3f} "
                  "samples f1: {:.3f}".format(epoch, iteration,
                                              result['micro/f1'],
                                              result['macro/f1'],
                                              result['samples/f1']))

            model.train()
        iteration += 1

    loss_value = np.mean(batch_losses)
    print("epoch:{:2d} iter:{:3d} train: loss:{:.3f}".format(epoch, iteration, loss_value))
    if epoch % save_freq == 0:
        checkpoint_save(model, save_path, epoch)
    epoch += 1
    if max_epoch_number < epoch:
        break

300